# Learning objectives
- Use `.query()` to filter data more cleanly
- Refresh `.groupby()` syntax for splitting and grouping data, and applying methods to grouped data
- Use `.agg()` to summarize grouped variables

In [1]:
import pandas as pd

# Make Google Drive available to the script
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
# Let's load the Oregon Health Insurance Experiment dataset
filename = 'drive/MyDrive/Colab Notebooks/Intro to Python for Epidemiologists/Data/OHIE_12m.csv'
OHIE = pd.read_csv(filename)
OHIE.head()

,person_id,household_id,treatment,draw_treat,draw_lottery,applied_app,approved_app,dt_notify_lottery,dt_retro_coverage,birthyear_list,...,live_partner_12m,live_parents_12m,live_friends_12m,live_relatives_12m,live_other_12m,hhsize_12m,PHQ2_1,PHQ2_2,PHQ2_sum,PHQ2_cutoff
0,64350,164350,Not selected,NaN,Lottery Draw 6,NaN,NaN,2008-07-14,2008-08-08,1974,...,No,Yes,No,No,No,2.0,3.0,3.0,6.0,True
1,55655,155655,Not selected,NaN,Lottery Draw 7,NaN,NaN,2008-08-12,2008-09-08,1987,...,Yes,No,No,No,No,2.0,1.0,1.0,2.0,False
2,20087,128134,Selected,Draw 6: selected in lottery 07/01/2008,Lottery Draw 6,Submitted an Application to OHP,No,2008-07-14,2008-08-08,1963,...,No,No,No,Yes,No,7.0,0.0,1.0,1.0,False
3,70998,170998,Not selected,NaN,Lottery Draw 7,NaN,NaN,2008-08-12,2008-09-08,1954,...,Yes,No,No,No,No,2.0,3.0,2.0,5.0,True
4,8839,108839,Selected,Draw 8: selected in lottery 09/02/2008,Lottery Draw 8,Did NOT submit an application to OHP,No,2008-09-11,2008-10-08,1964,...,No,No,Yes,No,No,4.0,2.0,2.0,4.0,True


# Advanced filtering with `.query()`

In [4]:
# Traditional filtering works with booleans:
OHIE[OHIE['birthyear_list'] < 1958]
OHIE.loc[OHIE['birthyear_list'] < 1958, ['person_id','treatment','birthyear_list']]

,person_id,treatment,birthyear_list
3,70998,Not selected,1954
7,7491,Selected,1952
16,8538,Selected,1951
19,37303,Selected,1946
20,23931,Not selected,1956
...,...,...,...
3979,60711,Selected,1952
3981,15291,Selected,1957
3982,50392,Not selected,1956
3983,72843,Not selected,1955


In [6]:
OHIE.query('birthyear_list < 1958').head()

,person_id,household_id,treatment,draw_treat,draw_lottery,applied_app,approved_app,dt_notify_lottery,dt_retro_coverage,birthyear_list,...,live_partner_12m,live_parents_12m,live_friends_12m,live_relatives_12m,live_other_12m,hhsize_12m,PHQ2_1,PHQ2_2,PHQ2_sum,PHQ2_cutoff
3,70998,170998,Not selected,NaN,Lottery Draw 7,NaN,NaN,2008-08-12,2008-09-08,1954,...,Yes,No,No,No,No,2.0,3.0,2.0,5.0,True
7,7491,107491,Selected,Draw 3: selected in lottery 04/08/2008,Lottery Draw 3,Submitted an Application to OHP,No,2008-04-16,2008-05-08,1952,...,No,No,No,No,No,1.0,0.0,0.0,0.0,False
16,8538,127696,Selected,Draw 7: selected in lottery 08/01/2008,Lottery Draw 7,Submitted an Application to OHP,No,2008-08-12,2008-09-08,1951,...,Yes,No,No,No,No,2.0,0.0,0.0,0.0,False
19,37303,158931,Selected,Draw 6: selected in lottery 07/01/2008,Lottery Draw 6,Did NOT submit an application to OHP,No,2008-07-14,2008-08-08,1946,...,Yes,No,No,No,No,2.0,0.0,0.0,0.0,False
20,23931,123931,Not selected,NaN,Lottery Draw 1,NaN,NaN,2008-03-10,2008-03-11,1956,...,No,No,No,Yes,No,3.0,2.0,2.0,4.0,True


In [10]:
# Query on multiple conditions
OHIE.query('birthyear_list < 1958 and treatment == "Selected"').head()
# OHIE.loc[(OHIE['birthyear_list'] < 1958) & (OHIE['treatment']=='Selected'),:]

,person_id,household_id,treatment,draw_treat,draw_lottery,applied_app,approved_app,dt_notify_lottery,dt_retro_coverage,birthyear_list,...,live_partner_12m,live_parents_12m,live_friends_12m,live_relatives_12m,live_other_12m,hhsize_12m,PHQ2_1,PHQ2_2,PHQ2_sum,PHQ2_cutoff
7,7491,107491,Selected,Draw 3: selected in lottery 04/08/2008,Lottery Draw 3,Submitted an Application to OHP,No,2008-04-16,2008-05-08,1952,...,No,No,No,No,No,1.0,0.0,0.0,0.0,False
16,8538,127696,Selected,Draw 7: selected in lottery 08/01/2008,Lottery Draw 7,Submitted an Application to OHP,No,2008-08-12,2008-09-08,1951,...,Yes,No,No,No,No,2.0,0.0,0.0,0.0,False
19,37303,158931,Selected,Draw 6: selected in lottery 07/01/2008,Lottery Draw 6,Did NOT submit an application to OHP,No,2008-07-14,2008-08-08,1946,...,Yes,No,No,No,No,2.0,0.0,0.0,0.0,False
27,15392,115392,Selected,Draw 4: selected in lottery 05/01/2008,Lottery Draw 4,Submitted an Application to OHP,No,2008-05-09,2008-06-09,1948,...,Yes,No,No,No,No,2.0,2.0,3.0,5.0,True
28,4104,104104,Selected,Draw 5: selected in lottery 06/02/2008,Lottery Draw 5,Submitted an Application to OHP,Yes,2008-06-11,2008-07-08,1956,...,No,No,Yes,No,No,1.0,2.0,1.0,3.0,True


# Using `.groupby()` to split data

In [11]:
# Grouping by sex
grouped = OHIE.groupby('female_list')
grouped

In [12]:
grouped['birthyear_list'].mean()

,birthyear_list
female_list,
0: Male,1964.982222
1: Female,1965.964124


In [15]:
grouped['treatment'].value_counts()

female_list  treatment   
0: Male      Not selected     788
             Selected         787
1: Female    Selected        1224
             Not selected    1201
Name: count, dtype: int64

# Summarizing grouped data with `.agg()`

In [21]:
# OHIE.loc[OHIE['treatment']=='Not selected','birthyear_list'].min()

In [23]:
# One summary variable:
OHIE.groupby('treatment').agg({
    'birthyear_list':['min','mean','max'],
    'PHQ2_sum':['mean','std']
})

birthyear_list                     PHQ2_sum          
                        min         mean   max      mean       std
treatment                                                         
Not selected           1945  1965.449975  1988  2.205325  2.005201
Selected               1945  1965.703630  1988  1.933574  1.935917

# Combining `.query()` with `.groupby()`

In [26]:
(OHIE
 .query('birthyear_list < 1970')
 .groupby('treatment')
 .agg({'PHQ2_sum':'mean'})
 )

,PHQ2_sum
treatment,
Not selected,2.420923
Selected,2.110549
